In [1]:
import asyncio
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

# ✅ GPT-4
llm = ChatOpenAI(model="gpt-4", temperature=0.2)

# ✅ 프롬프트 구성
gt_prompt = PromptTemplate.from_template("""
아래는 '{question}'에 대한 질문이야. 
가능하면 정확하고 간단한 정보로, 2~3문장으로 요약해서 알려줘.
시니어도 이해하기 쉬운 말투로 알려줘.

질문: {question}

답변:
""")

# ✅ 체인 구성
gt_chain = gt_prompt | llm

# ✅ 질문 템플릿 & 인물 리스트
person_list = [ "임영웅", "이찬원", "정동원", "송가인", "김연자" ]  # 예시 (전체 쓰면 오래 걸림)
question_templates = [
    "{}는 몇 살이야?",
    "{} 생일이 언제야?",
    "{}의 본명은 뭐야?",
    "{}는 어디 소속이야?",
    "{}의 대표곡 알려줘",
    "{}는 어디 출신이야?",
    "{} 키 몇이야?",
    "{}의 데뷔년도는?",
    "{} 어떤 장르야?",
    "{} 최근 활동 뭐 있어?"
]

# ✅ 질문 생성
sample_questions = []
for name in person_list:
    for template in question_templates:
        sample_questions.append(template.format(name))

# ✅ Ground Truth 생성
async def generate_gt_answer(question: str):
    try:
        response = await gt_chain.ainvoke({"question": question})
        return response.content.strip()
    except Exception as e:
        print(f"❌ 오류 발생 ({question}): {e}")
        return None

# ✅ 하나의 샘플 생성
async def build_sample(question: str):
    answer = await generate_gt_answer(question)
    if answer:
        print(f"✅ 질문: {question}\n➡️ 답변: {answer}\n" + "-"*50)
        return {"question": question, "ground_truth": answer}
    return None

# ✅ 전체 실행
async def generate_gt_dataset(questions: list[str]):
    print("🚀 인물 질문 Ground Truth 생성 시작...\n")
    results = await asyncio.gather(*[build_sample(q) for q in questions])
    valid = [r for r in results if r]

    df = pd.DataFrame(valid)
    df.to_excel("person_gt_dataset.xlsx", index=False)
    print("\n📁 저장 완료: person_gt_dataset.xlsx")

In [2]:
await generate_gt_dataset(sample_questions)

🚀 인물 질문 Ground Truth 생성 시작...

✅ 질문: 정동원 생일이 언제야?
➡️ 답변: 정동원의 생일은 1989년 3월 23일이에요.
--------------------------------------------------
✅ 질문: 임영웅의 본명은 뭐야?
➡️ 답변: 임영웅의 본명은 '임영웅'입니다. 그는 본명을 그대로 가수 이름으로 사용하고 있어요.
--------------------------------------------------
✅ 질문: 김연자 생일이 언제야?
➡️ 답변: 김연자의 생일은 1968년 12월 5일이야.
--------------------------------------------------
✅ 질문: 이찬원는 어디 출신이야?
➡️ 답변: 이찬원은 대한민국 경상북도 안동시 출신입니다.
--------------------------------------------------
✅ 질문: 이찬원의 본명은 뭐야?
➡️ 답변: 이찬원의 본명은 '이찬원'입니다. 그는 본명 그대로 활동하고 있는 가수입니다.
--------------------------------------------------
✅ 질문: 이찬원 생일이 언제야?
➡️ 답변: 이찬원의 생일은 1996년 12월 2일이에요.
--------------------------------------------------
✅ 질문: 송가인 키 몇이야?
➡️ 답변: 송가인의 키는 공식적으로 알려진 바가 없습니다. 그러나 대략적으로 160cm 정도로 추정되고 있습니다.
--------------------------------------------------
✅ 질문: 송가인는 몇 살이야?
➡️ 답변: 송가인은 1986년 3월 22일에 태어났으므로, 2022년 현재로서 36세입니다.
--------------------------------------------------
✅ 질문: 임영웅 키 몇이야?
➡️ 답변: 임영웅의 키는 약 175cm입니다. 이는 평균적인 한국 남